In [36]:
from config import MY_NAME, ES_SERVER

from elasticsearch import Elasticsearch
es = Elasticsearch(ES_SERVER)

`wget http://www.statmt.org/wmt14/training-monolingual-news-crawl/news.2013.de.shuffled.gz`

In [33]:
INDEX = MY_NAME + '_news2013'

es.indices.delete(index=INDEX, ignore=[400, 404])

with open('news.2013.de.shuffled_500k') as f:
    for i, txt in enumerate(f):
        es.index(index=INDEX, doc_type='news', body={'mytext': txt.rstrip()})
        if i % 500 == 0:
            print(i)
        if i > 3000:
            break
            
es.count(index=INDEX, doc_type='news')

0
500
1000
1500
2000
2500
3000


KeyboardInterrupt: 

In [47]:
q = {  
    "query": { 
        "query_string": { 
            "query": "Marcel Rodman" 
        }
    }
}
# match phrase
# https://www.elastic.co/guide/en/elasticsearch/reference/6.2/query-dsl-match-query-phrase.html
q2 = {  
    "query": { 
        "match_phrase": { 
            "mytext": "Marcel Rodman" 
        }
    }
}


for hit in es.search(index=INDEX, body = q)['hits']['hits']:
    print(round(hit['_score'],2), hit['_source']['mytext'])


14.3 Ebenfalls neu im Team der Wild Wings ist Marcel Rodman.
8.19 Marcel Koller über...
7.09 Und die Beschreibungen stimmten überein!", sagt Marcel.
5.08 "Wir konnten unsere körperliche Überlegenheit ausnutzen", freute sich Marchefka über das frühe Tor von Marcel Runge, der per Distanzschuss getroffen hatte (10.).


# Highlight

In [61]:
q = {
    "query" : {
        "match": { "mytext": "Marcel" }
    },
    "highlight" : {
        "pre_tags" : ["<strong>"],
        "post_tags" : ["</strong>"],
        "fields" : {
            "mytext" : {}
        }
    }
}

from IPython.core.display import HTML
from IPython.core.display import display, HTML
for hit in es.search(index=INDEX, body = q)['hits']['hits']:
    display(HTML('{} {}'.format(round(hit['_score'],2), hit['highlight']['mytext'][0])))


